In [ ]:
!nvidia-smi

Thu Dec 12 16:55:36 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   77C    P0              34W /  70W |   2175MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf

print("TensorFlow version:", tf.__version__)
print("Is GPU available:", tf.test.is_gpu_available())
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


TensorFlow version: 2.17.1
Is GPU available: True
Num GPUs Available:  1


In [ ]:
!pip uninstall tensorflow -y
!pip install tensorflow==2.11.0


Found existing installation: tensorflow 2.17.1
Uninstalling tensorflow-2.17.1:
  Successfully uninstalled tensorflow-2.17.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 71.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 59.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 103.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 101.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 45.6 MB/s eta 0:00:00
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.7.2
    Uninstalling tensorboard-data-server-0.7.2:
      Successfully uninstalled tensorboard-data-server-0.7.2
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.5
    Uninstalling proto

In [ ]:
!pip install tensorflow-gpu==2.11.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 2.5 MB/s eta 0:00:00


In [ ]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
#from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
#import matplotlib.pyplot as plt

Resize to fit for inception v3 model

In [ ]:
IMAGE_SIZE = [224, 224]

train_path = '/content/drive/MyDrive/cnn/train'
valid_path = '/content/drive/MyDrive/cnn/test'

Here we will be using imagenet weights

In [ ]:
inception = InceptionV3(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

Freeze the layers of InceptionV3 to prevent them from being trained

In [ ]:
for layer in inception.layers:
    layer.trainable = False

In [ ]:
folders = glob('/content/drive/MyDrive/cnn/train/*')

In [ ]:
folders

['/content/drive/MyDrive/cnn/train/Potato___healthy',
 '/content/drive/MyDrive/cnn/train/Tomato_Early_blight',
 '/content/drive/MyDrive/cnn/train/Potato___Late_blight',
 '/content/drive/MyDrive/cnn/train/Potato___Early_blight',
 '/content/drive/MyDrive/cnn/train/Tomato_healthy',
 '/content/drive/MyDrive/cnn/train/Tomato_Late_blight',
 '/content/drive/MyDrive/cnn/train/unknown']

In [ ]:
x = Flatten()(inception.output)

Use softmax rather than relu for multiclass classification
Dense layer with nodes equal to number of classes

In [ ]:
prediction = Dense(len(folders), activation='softmax')(x)

In [ ]:
model = Model(inputs=inception.input, outputs=prediction)

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 111, 111, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 111, 111, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                             

In [ ]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

Data augmentation

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/train',
                                                 target_size = (224, 224),
                                                 batch_size = 16,
                                                 class_mode = 'categorical')

Found 5330 images belonging to 7 classes.


In [ ]:
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/test',
                                            target_size = (224, 224),
                                            batch_size = 16,
                                            class_mode = 'categorical')

Found 1423 images belonging to 7 classes.


In [ ]:
r = model.fit(
  training_set,
  validation_data=test_set,
  epochs=13,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

Epoch 1/13
334/334 [==============================] - 688s 2s/step - loss: 2.3715 - accuracy: 0.7946 - val_loss: 1.7530 - val_accuracy: 0.8503
Epoch 2/13
334/334 [==============================] - 557s 2s/step - loss: 1.4191 - accuracy: 0.8827 - val_loss: 1.1183 - val_accuracy: 0.9023
Epoch 3/13
334/334 [==============================] - 560s 2s/step - loss: 1.6173 - accuracy: 0.8876 - val_loss: 1.5760 - val_accuracy: 0.8960
Epoch 4/13
334/334 [==============================] - 561s 2s/step - loss: 1.3968 - accuracy: 0.9069 - val_loss: 3.0774 - val_accuracy: 0.8517
Epoch 5/13
334/334 [==============================] - 522s 2s/step - loss: 0.9498 - accuracy: 0.9375 - val_loss: 1.5557 - val_accuracy: 0.9100
Epoch 6/13
334/334 [==============================] - 511s 2s/step - loss: 1.4504 - accuracy: 0.9184 - val_loss: 2.4479 - val_accuracy: 0.8700
Epoch 7/13
334/334 [==============================] - 512s 2s/step - loss: 0.9203 - accuracy: 0.9420 - val_loss: 1.1871 - val_accuracy: 0.9368

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from tensorflow.keras.models import load_model

model = load_model('/content/drive/MyDrive/cnn/model_inception.h5')

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

test_datagen = ImageDataGenerator(rescale=1./255)

test_set = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/Test',
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical',
    shuffle=False
)


Found 1345 images belonging to 7 classes.


Make predictions using the reloaded model

In [ ]:
y_pred = model.predict(test_set)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


85/85 ━━━━━━━━━━━━━━━━━━━━ 608s 7s/step


Convert probabilities to predicted class indices

In [ ]:
import numpy as np
predicted_classes = np.argmax(y_pred, axis=1)

Get the true class labels from the generator

In [ ]:
true_classes = test_set.classes


In [ ]:
from sklearn.metrics import accuracy_score, classification_report

accuracy = accuracy_score(true_classes, predicted_classes)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

report = classification_report(true_classes, predicted_classes, target_names=test_set.class_indices.keys())
print(report)


Test Accuracy: 93.31%
                       precision    recall  f1-score   support

Potato___Early_blight       0.93      0.98      0.96       200
 Potato___Late_blight       0.85      0.95      0.90       200
     Potato___healthy       1.00      0.68      0.81        31
  Tomato_Early_blight       0.94      0.84      0.89       200
   Tomato_Late_blight       0.94      0.93      0.93       382
       Tomato_healthy       0.98      0.97      0.98       319
                 unkn       0.93      1.00      0.96        13

             accuracy                           0.93      1345
            macro avg       0.94      0.91      0.92      1345
         weighted avg       0.94      0.93      0.93      1345

